In [29]:
import os
import shutil
import pandas as pd
import datetime as dt
import numpy as np
import re
import datetime

In [30]:
task_path = r'Z:\pdn\data\task'

In [31]:
def get_filenames(path: str, patterns: list, exclude_keys: list):
    
    print(patterns)
    
    all_files = os.listdir(path)
    out = all_files.copy()
    for pattern in patterns:
        for file in all_files:
            if pattern not in file:
                out.remove(file)
        all_files = out.copy()
    
    for key in exclude_keys:
        for file in all_files:
            if key in file:
                out.remove(file)  
        all_files = out.copy()
    
    out = [os.path.join(path, f) for f in out]
    return out

In [32]:
def get_concated_task(
        task_path: str,
        date: datetime.datetime,
        patterns: list = ['part'],
        exclude_keys: list = ['~$']
):
    # Загружает emias_data за указанную дату
    # исключает из списка загружаемых таблиц те,
    # в названии которых есть элемент из exclude_types
    # возвращает конкатенацию всех файлов, если concat == True
    # иначе возвращает словарь, в котором месяц/тип  таблицы является ключём

    file_list = get_filenames(task_path, patterns = [date.strftime('%Y%m%d'), *patterns], exclude_keys=exclude_keys)

    print(file_list)
                
    df = pd.DataFrame()
    for path in file_list:
        if df.empty:
            df = pd.read_excel(path)
        else:
            df = pd.concat([df, pd.read_excel(path)])
    
    return df
        

In [33]:
def create_task_file(month, date, df, task_path):
    '''Выгрузка ПОСЛЕДНЕЙ ВЕРСИИ задания из папки paths.TASK_SUBFOLDER_LOADS в
    объект DataFrame'''
    date_ymd = date.strftime('%Y%m%d')

    file_name = f'{month}_{date_ymd}_v20.xlsx'
    df.to_excel(file_name, index=False)
    return file_name

In [34]:
def main_giga_func(
    months: list, 
    date = datetime.datetime.today(), 
    task_path = task_path,
    concat=True
):
    
    giga_task = []
    for month in months:
        task = get_concated_task(
            task_path=task_path,
            date=date,
            patterns = ['part', month]
        )
        file_name = create_task_file(month, date, task, task_path=task_path)  
        shutil.move(
            os.path.join(os.getcwd(), file_name),
            os.path.join(task_path, file_name)
        ) 
        
        giga_task.append(task)
    
    if concat:
        giga_task = pd.concat(giga_task)
        file_name = create_task_file('fulltask', date, giga_task, task_path=task_path)
        shutil.move(
            os.path.join(os.getcwd(), file_name),
            os.path.join(task_path, file_name)
        )

In [35]:
main_giga_func(months=['september', 'august'])

['20220920', 'part', 'september']
['Z:\\pdn\\data\\task\\september_20220920_part_1.xlsx', 'Z:\\pdn\\data\\task\\september_20220920_part_2.xlsx']
['20220920', 'part', 'august']
['Z:\\pdn\\data\\task\\august_20220920_part_1.xlsx', 'Z:\\pdn\\data\\task\\august_20220920_part_2.xlsx']


In [36]:
datetime.datetime.now()

datetime.datetime(2022, 9, 20, 11, 35, 25, 840976)